---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [16]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_towns = open("university_towns.txt", "r")
    
    university_towns_array = []
    
    for obj in university_towns:
        if 'edit' in obj:
            state = obj[0:obj.find('[')]
            
        else:
            town = obj[0:obj.find('(')-1]
            university_towns_array.append([state,town])
        
    
    return university_towns_array

print(get_list_of_university_towns())

[['Alabama', 'Auburn'], ['Alabama', 'Florence'], ['Alabama', 'Jacksonville'], ['Alabama', 'Livingston'], ['Alabama', 'Montevallo'], ['Alabama', 'Troy'], ['Alabama', 'Tuscaloosa'], ['Alabama', 'Tuskegee'], ['Alaska', 'Fairbanks'], ['Arizona', 'Flagstaff'], ['Arizona', 'Tempe'], ['Arizona', 'Tucson'], ['Arkansas', 'Arkadelphia'], ['Arkansas', 'Conway'], ['Arkansas', 'Fayetteville'], ['Arkansas', 'Jonesboro'], ['Arkansas', 'Magnolia'], ['Arkansas', 'Monticello'], ['Arkansas', 'Russellville'], ['Arkansas', 'Searcy'], ['California', 'Angwin'], ['California', 'Arcata'], ['California', 'Berkeley'], ['California', 'Chico'], ['California', 'Claremont'], ['California', 'Cotati'], ['California', 'Davis'], ['California', 'Irvine'], ['California', 'Isla Vista'], ['California', 'University Park, Los Angeles'], ['California', 'Merced'], ['California', 'Orange'], ['California', 'Palo Alto'], ['California', 'Pomona'], ['California', 'Redlands'], ['California', 'Riverside'], ['California', 'Sacramento']

In [5]:
# Make it easy to call for multiple questions:

def read_gdp():
    
    GDP = pd.read_excel('gdplev.xls', skiprows=4) # Drop irrelevant rows.
    
    GDP = GDP.drop(GDP.columns[[0,1,2,3,5,7]],axis=1) # Remove unnecessary data.
    
    # Setting the first row to header:
    new_header = GDP.iloc[0]
    GDP = GDP[3:]
    GDP.columns = new_header
    
    # More cleaning:
    GDP = GDP.reset_index(drop=True)
    GDP.columns = ['Quarter','GDP']
    
    # Remove data before 2000 Q1:
    GDP = GDP.drop(GDP.index[0:212])
    #return GDP.index[GDP['Quarter'] == '2000q1'] # Returns index = 212 for 2000q1.
    
    GDP = GDP.reset_index(drop=True)
    
    GDP['GDP Diff'] = GDP['GDP'].diff() # Finds the difference between successive row entries in the column 'GDP'.
    
    return GDP

print(read_gdp())

   Quarter      GDP GDP Diff
0   2000q1  12359.1      NaN
1   2000q2  12592.5    233.4
2   2000q3  12607.7     15.2
3   2000q4  12679.3     71.6
4   2001q1  12643.3      -36
5   2001q2  12710.3       67
6   2001q3  12670.1    -40.2
7   2001q4  12705.3     35.2
8   2002q1  12822.3      117
9   2002q2    12893     70.7
10  2002q3  12955.8     62.8
11  2002q4    12964      8.2
12  2003q1  13031.2     67.2
13  2003q2  13152.1    120.9
14  2003q3  13372.4    220.3
15  2003q4  13528.7    156.3
16  2004q1  13606.5     77.8
17  2004q2  13706.2     99.7
18  2004q3  13830.8    124.6
19  2004q4  13950.4    119.6
20  2005q1  14099.1    148.7
21  2005q2  14172.7     73.6
22  2005q3  14291.8    119.1
23  2005q4  14373.4     81.6
24  2006q1  14546.1    172.7
25  2006q2  14589.6     43.5
26  2006q3  14602.6       13
27  2006q4  14716.9    114.3
28  2007q1    14726      9.1
29  2007q2  14838.7    112.7
..     ...      ...      ...
36  2009q1    14375     -202
37  2009q2  14355.6    -19.4
38  2009q3  14

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    
    ''' recession is defined as starting with two consecutive quarters of GDP 
    decline, and ending with two consecutive quarters of GDP growth'''
    
    gdp_df = read_gdp()
    
    buf = 0
    count = 0
    
    for i in gdp_df['GDP Diff']:
        
        if i < 0 and buf <0:
            return gdp_df[gdp_df['GDP Diff'] == gdp_df['GDP Diff'].iloc[count - 1]].iloc[0].loc['Quarter']
            
        count += 1
        buf = i

get_recession_start()

'2008q3'

In [7]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = read_gdp()
    
    start_of_rec_index = gdp_df[gdp_df['Quarter'] == get_recession_start()].index[0]
    
    gdp_df = gdp_df.drop([i for i in range(0,start_of_rec_index + 2)])
    
    buf = 0
    count = 0
    
    for i in gdp_df['GDP Diff']:
        
        if i > 0 and buf > 0:
            return gdp_df[gdp_df['GDP Diff'] == gdp_df['GDP Diff'].iloc[count]].iloc[0].loc['Quarter']
            
        count += 1
        buf = i

print(get_recession_end())

2009q4


In [8]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    gdp_df = read_gdp()
    
    start_of_rec_index = gdp_df[gdp_df['Quarter'] == get_recession_start()].index[0]
    end_of_rec_index = gdp_df[gdp_df['Quarter'] == get_recession_end()].index[0]
    
    ranges = [i for i in range(0,start_of_rec_index)] + [i for i in range(end_of_rec_index,len(gdp_df))]
    
    gdp_df = gdp_df.drop(ranges)
    
    return gdp_df[gdp_df['GDP'] == gdp_df['GDP'].min()].iloc[0]['Quarter']

print(get_recession_bottom())

2009q2


In [9]:
def change_to_quarter(date: str):
    date = date.split('-')
    month = int(date[1])
    quarter = int((month - 1) / 3) + 1
    return date[0] + 'q' + str(quarter)


print(change_to_quarter('1996-04'))

1996q2


In [14]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
    all_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
    
    # Drop columns with unnecessary data:
    start_rem = all_homes.columns.get_loc('1996-04')
    end_rem = all_homes.columns.get_loc('2000-01')
    all_homes = all_homes.drop(all_homes.columns[start_rem:end_rem],axis=1) 
    
    # Double square brackets if you're giving specific colums to remove, single square brackets if it's a column range.
    
    # Removing more unnecessary columns:
    all_homes = all_homes.drop(all_homes.columns[[0,3,4,5]],axis=1)
    
    # Map state short forms with given dictionary:
    all_homes['State'] = all_homes['State'].map(states)
    
    # Switch State and RegionName columns for multiindex:
    columnsName = list(all_homes.columns)
    S, R = columnsName.index('State'), columnsName.index('RegionName')
    columnsName[S], columnsName[R] = columnsName[R],columnsName[S]
    all_homes = all_homes[columnsName]
    
    # Sorts and groups by index:
    all_homes = all_homes.set_index(['State','RegionName']).sort_index()
    
    # Group by user defined function (above) which changes given dates to year + quarter. Axis = 1 specifies you're passing column names to the function.
    all_homes = all_homes.groupby(change_to_quarter, axis=1).mean() # Find mean over the months in a quarter.
    
    return all_homes

print(convert_housing_data_to_quarters())

                                  2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.6666

In [120]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    df = convert_housing_data_to_quarters()
    
    # Start position is the quarter BEFORE the recession starts!
    before_rec = (df.columns.get_loc(get_recession_start())-1)
    rec_bottom = df.columns.get_loc(get_recession_bottom())
    
    university = get_list_of_university_towns()
    
    universities_array = np.array(university)
    
    universities_dataframe = pd.DataFrame({'State': universities_array[:, 0], 'RegionName': universities_array[:, 1]})
    
    uni = universities_dataframe.set_index(['State', 'RegionName'])
    
    
    # Turn the divided values into a DataFrame!
    df = np.divide(df.ix[:,before_rec],df.ix[:,rec_bottom]).to_frame().dropna()
    
    # Merge university and GDP data.
    uni_df = df.merge(uni, right_index=True, left_index=True, how='inner')
    
    # Drop the indices of uni towns to get data only for non uni towns.
    nonuni_df = df.merge(uni, right_index=True, left_index=True, how='left')
    nonuni_df = df.drop(uni_df.index)

    
    
    
    p_value = ttest_ind(uni_df.values, nonuni_df.values).pvalue
    
    if p_value < 0.1:
        p = True
        
    else:
        p = False
        
    if float(nonuni_df.mean()) < float(uni_df.mean()):
        better = 'non-university town'
        
    else:
        better = 'university town'

    return (p, p_value[0], better)
    
run_ttest()

(True, 0.002724063704761164, 'university town')